In [ ]:
%pip install -r requirements.txt

In [10]:
from openai import AzureOpenAI
import os
import pgvector
import numpy as np
import psycopg
from pgvector.psycopg import register_vector
import json
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from dotenv import find_dotenv, load_dotenv


In [7]:
# Set up the token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_version="2024-04-01-preview",
    azure_endpoint="https://oai-use2-dcsvc-np-ssv.openai.azure.com/",
    azure_ad_token_provider=token_provider
)

# Use the text embedding model
response = client.embeddings.create(
    model="text-embedding-ada-002",  # model = "deployment_name"
    input="Your text here"
)

# Print the embedding vector
print(response.data[0].embedding)


[-0.0119464211165905, -0.004136581905186176, 0.017172981053590775, -0.017546307295560837, -0.023146191611886024, 0.028986072167754173, -0.015986338257789612, -0.013426391407847404, -0.01236641313880682, -0.011353100650012493, -0.0018382956041023135, 0.009119812399148941, -0.011819757521152496, -0.006406535394489765, 0.012206416577100754, -0.006879858672618866, 0.02339952066540718, 0.00132163951639086, 0.017692970111966133, -0.014013045467436314, -0.007846505381166935, 0.01114643830806017, 0.001391638070344925, -0.0034099300391972065, -0.0033332649618387222, -0.0025032819248735905, 0.01883961260318756, -0.025186149403452873, 0.011459765024483204, 0.0017716302536427975, 0.010946442373096943, -0.009006481617689133, -0.012586408294737339, -0.011559762991964817, -0.015599680133163929, -0.005316557362675667, -0.0017032984178513288, -0.010346454568207264, 0.032159339636564255, -0.018466288223862648, 0.02017292007803917, 0.015266353264451027, 0.0022316209506243467, 0.00011312268179608509, -0.0

In [ ]:
# Set up the database connection
load_dotenv(find_dotenv(), override=True)

DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD', '') 
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')

# Make sure password is not None
if DB_PASSWORD is None:
    DB_PASSWORD = ''

# Troubleshooting connection
connection_string = f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} host={DB_HOST} port={DB_PORT}"
print(f"Attempting to connect with: {connection_string.replace(DB_PASSWORD, 'pf832yqsh0zcECMd9s9C8EwU01hpix' if DB_PASSWORD else 'no password')}")

try:
    conn = psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
    print("Connection successful!")
    conn.close()
except psycopg.OperationalError as e:
    print(f"Connection failed: {e}")
    # Don't exit here to allow for debugging

Attempting to connect with: dbname=postgres user=psqladmin password=pf832yqsh0zcECMd9s9C8EwU01hpix host=psql-use2-dcsvc-np-mns-dev-virtual-vendor.postgres.database.azure.com port=5432
Connection successful!


In [50]:
# Set up the database connection
load_dotenv(find_dotenv(), override=True)

DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')

try:
    conn = psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
    conn.close()
except psycopg.OperationalError as e:
    exit(1)

# Fetch data from table and generate embeddings
def fetch_data_and_generate_embeddings(table_name):
    with psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as conn:
        with conn.cursor() as cur:
            cur.execute(f"SELECT * FROM {table_name}")
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]

    # Generate embeddings for each row
    embeddings = []
    for row in rows:
        text = " ".join(map(str, row))  # Convert row to string
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=text
        )
        embeddings.append(response.data[0].embedding)

    # return embeddings
    # Save embeddings to the database
    with psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as conn:
        with conn.cursor() as cur:
            for i, embedding in enumerate(embeddings):
                cur.execute(f"UPDATE {table_name} SET embedding = %s WHERE id = %s", (json.dumps(embedding), rows[i][0]))
            conn.commit()

# === MAIN LOGIC ===
if __name__ == "__main__":
    fetch_data_and_generate_embeddings("listings")

    # Fetch data inserted into the database
    with psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as conn:
        with conn.cursor() as cur:
            cur.execute(f"SELECT * FROM listings")
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]

    # Print the fetched data
    for row in rows:
        print(dict(zip(columns, row)))

{'id': 22, 'name': 'Apt on Lake Union', 'description': "Cookbook authors rad, little 1 bedroom apt overlooking Lake Union. Amazing east-facing sunrise view of mountains & lake. Full herb garden (while it lasts) & well-appointed kitchen for cooking & baking during your stay and a well-stocked urban pantry! Rad little one bedroom apt overlooking Lake Union. Amazing east-facing sunrise view of mountains & lake. Full herb garden (while it lasts) & well-appointed kitchen for cooking & baking during your stay. Well-stocked urban pantry full of handmade preserves, too! Everything in the pantry is whole food, organic and handmade and you're welcome to cook something up for yourself whilst here - no need to go grocery shopping for basics during your stay. Queen bed with organic cotton sheets. Apartment has wifi, HBO, Apple TV + local cable TV & a small desk with monitor.", 'property_type': 'Apartment', 'room_type': 'Entire home/apt', 'price': Decimal('110.00'), 'weekly_price': Decimal('692.00')

In [52]:
# Generate embedding for query data and compare with database to return the most similar
# query_data = "shining and blazing"
# query_data = "bright natural light"
query_data = "wholefoods nearby"

def generate_embeddings_for_query_data(query_data):
    embeddings = []
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=query_data
    )
    embeddings.append(response.data[0].embedding)
    return embeddings

# Check embedding with database
def fetch_similarity_data_from_db(table_name):
    with psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as conn:
        # Register the vector type with psycopg
        register_vector(conn)
        
        with conn.cursor() as cur:
            # First, convert the text query to an embedding vector
            query_embedding = generate_embeddings_for_query_data(query_data)[0]
            print(f"Query embedding: {query_embedding}")
            
            # Then use the vector similarity search in the ORDER BY clause
            cur.execute(
                f"""
                SELECT * FROM {table_name} 
                ORDER BY embedding <=> %s::vector
                LIMIT 1
                """, 
                (query_embedding,)
            )
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
    return rows, columns

# === MAIN LOGIC ===
if __name__ == "__main__":
    rows, columns = fetch_similarity_data_from_db("listings")

    # Print the fetched data
    for row in rows:
        print(dict(zip(columns, row)))

Query embedding: [-0.0006340531981550157, -0.02509191818535328, -0.021534720435738564, 0.016294697299599648, -0.028840655460953712, 0.007312775123864412, -0.006916011217981577, -0.018538467586040497, 0.02680210769176483, -0.015364352613687515, 0.018141701817512512, -3.644843673100695e-05, -0.02094641514122486, -0.02139790542423725, -0.009720724076032639, 0.014529779553413391, 0.029661547392606735, -0.006160106975585222, 0.004477279260754585, -0.04479331523180008, -0.014050926081836224, 0.029059559106826782, 0.013120582327246666, -0.019646670669317245, -0.003187795402482152, 0.014557142741978168, 0.015186493285000324, -0.020741192623972893, -0.023053370416164398, -0.0024370215833187103, 0.03885553032159805, -0.015925295650959015, -0.023669037967920303, 0.005082686431705952, 0.0010816955473273993, -0.004887724760919809, 0.006108800880610943, -0.02160312794148922, 0.03880080580711365, -0.02217775210738182, -0.0024353114422410727, -0.006249036639928818, 0.003988164477050304, -0.02386057935

In [ ]:
# Generate embedding for query data and compare with database to return the most similar
# query_data = "shining and blazing"
# query_data = "bright natural light"
query_data = "pet friendly"

def generate_embeddings_for_query_data(query_data):
    embeddings = []
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=query_data
    )
    embeddings.append(response.data[0].embedding)
    return embeddings

# Check embedding with database
def fetch_similarity_data_from_db(table_name):
    with psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as conn:
        # Register the vector type with psycopg
        register_vector(conn)
        
        with conn.cursor() as cur:
            # First, convert the text query to an embedding vector
            query_embedding = generate_embeddings_for_query_data(query_data)[0]
            print(f"Query embedding: {query_embedding}")
            
            # Then use the vector similarity search in the ORDER BY clause
            cur.execute(
                f"""
                SELECT name, description, property_type, room_type, price 
                FROM listings
                where price > '100.00' and price < '250.00'
                ORDER BY embedding <=> %s::vector
                LIMIT 3
                """, 
                (query_embedding,)
            )
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
    return rows, columns

# === MAIN LOGIC ===
if __name__ == "__main__":
    rows, columns = fetch_similarity_data_from_db("listings")

    # Print the fetched data
    for row in rows:
        print(dict(zip(columns, row)))